In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

import time
import os

import pandas as pd

import xlwings as xw

In [ ]:
company_list = pd.read_excel("Company Screening Report_aus.xls")

In [ ]:
company_list.head()

In [ ]:
transcripts_Summary_href = "https://www.capitaliq.com/CIQDotNet/Transcripts/Summary.aspx"

In [ ]:
def getBrowser(username=None, password=None):
    # start web browser
    browser=webdriver.Chrome()
    browser.get(transcripts_Summary_href)

    browser.find_element_by_xpath('//*[@id="username"]').send_keys(username)
    browser.find_element_by_xpath('//*[@id="password"]').send_keys(password)
    browser.find_element_by_xpath('//*[@id="myLoginButton"]').click()
    
    return browser

In [ ]:
def containing_Quarter(file_name):
    return ('Q1' in file_name) or ('Q2' in file_name) or ('Q3' in file_name) or ('Q4' in file_name)

In [ ]:
def getHTML(browser, startID=0, endID=company_list.shape[0], IDs = []):
    
    app=xw.App(visible=True,add_book=False)
    # 將<path替換為你的檔案路徑>
    wblist = app.books.open('<path>.xls')
    shtlist = wblist.sheets[0]

    first_half_XPath = '//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr['
    last_half_XPath = ']/td[3]/span/a'
    
    #for i in IDs:  # 自己選擇要爬哪些ID
    for i in range(startID, endID):

        if not company_list["transcript"][i]:
            continue
        
        shtlist.cells(i+2,1).value = 1

        # Get the IQID and ID of current company
        #
        # e.g. IQID of company "3Com Corporation" is IQ24107
        #      ID   of company "3Com Corporation" is 3
        #
        company_IQID = company_list["ExcelCompanyID"][i][2:]
        company_ID = company_list["id"][i]

        # Get the href of summary of all transcripts of current company
        #
        # e.g. https://www.capitaliq.com/CIQDotNet/Transcripts/Summary.aspx?companyId=24107
        #
        company_href = transcripts_Summary_href + "?companyId=" + str(company_IQID)

        # Go to that href
        browser.get(company_href)
        time.sleep(8)
        
        # Recheck if this company has no transcript
        table_length = len(browser.find_elements_by_xpath('//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr'))    
        if table_length == 1:
            shtlist.cells(i+2,1).value = 0
            continue

        # Get the total number of transcript 
        #
        # e.g. Extract "14" from "Viewing 1-14 of 14"
        #
        all_trans_numbers = None
        try:
            all_trans_numbers_XPath = first_half_XPath + str(table_length) + ']/td'
            all_trans_numbers = browser.find_element_by_xpath(all_trans_numbers_XPath).text
            all_trans_numbers = int(all_trans_numbers.split()[3])
        except:
            shtlist.cells(i+2,1).value = 0
            print(company_ID, "found no number")
            continue

        # Click 'View All' botton if it exist
        try:
            browser.find_element_by_link_text('View All').click()

            # 有時候 View All 還沒跑完就開始拉資料了，所以等一下，
            # 因為 transcripts 越多就要等越久，
            # 所以這裡我設需要等 (all_trans_numbers / 10) 秒
            time.sleep(all_trans_numbers / 10)
        except:
            pass


        # Get all of the hrefs and names of transcipts of company
        all_transcipts_href = []
        all_transcipts_name = []
        table_length = len(browser.find_elements_by_xpath('//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr'))
        for j in range(2, table_length):

            full_XPath = first_half_XPath + str(j) + last_half_XPath

            try:
                page_href = browser.find_element_by_xpath(full_XPath).get_attribute('href')
                name = browser.find_element_by_xpath(full_XPath).text
                
                if(containing_Quarter(name)):
                    all_transcipts_name.append(name)
                    all_transcipts_href.append(page_href)
            except:
                pass

        #print(all_trans_numbers, len(all_transcipts_href))
        
        # Check for missing transcipts 
        # 因為有些 transcipts 還沒被放上去只有文字標題，
        # 但是它還是被算在 all_trans_numbers 裡面，
        # 所以選擇 >10 確保沒有遺失 transcipts
        if all_trans_numbers - len(all_transcipts_href) > 10:
            print("not retrieve all trans in id :", company_ID)

        filepath = os.path.join('transcripts', str(company_ID))
        # Check if the folder exists. If not, create one
        if not os.path.exists(filepath):
            os.mkdir(filepath)

        for j in range(len(all_transcipts_href)):

            browser.get(all_transcipts_href[j])
            html = browser.page_source

            filename = os.path.join(filepath, all_transcipts_name[j].replace('/', '_')) + '.html'
            #print(filename)
            
            try:
                f = open(filename, "w", encoding="utf-8")
                f.write(str(BeautifulSoup(html, 'html.parser').prettify()))
                f.close()

            except:
                f.close()
                pass
            
        if len(all_transcipts_href) != len(os.listdir(filepath)):
            print(len(all_transcipts_href) - len(os.listdir(filepath)), \
                    "transcript missed for id :", company_ID)

In [ ]:
browser = getBrowser(username = "<your account>", password = "<your password>")

# Waiting for logging
time.sleep(10)

getHTML(browser)